# ПРОЕКТ 1 "Исследование надёжности заёмщиков"

Исходные данные:
По заданию кредитного отдела банка на основе статистики платежеспособности клиентов необходимо определить, влияет ли семейное положение и количество детей на возвращаемость кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга.

## Состав проекта

1.   [Шаг 1. Изучение исходных данных](#prestudy)
2.   [Шаг 2. Предобработка данных](#prep)<br>
  2.1. Обработка пропусков<br>
  2.2. Замена типа данных<br>
  2.3. Обработка дуликатов<br>
  2.4. Лемматизация<br>
  2.5. Исправление ошибок в данных<br>
  2.6. Категоризация данных<br>

3.   [Шаг 3. Анализ данных](#analysis)<br>
  3.1: Есть ли зависимость между наличием детей и возвратом кредита в срок?<br>
  3.2: Есть ли зависимость между семейным положением и возвратом кредита в срок?<br>
  3.3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<br>
  3.4: Как разные цели кредита влияют на его возврат в срок?<br>

4.   [Шаг 4. Общий вывод](#resume)

## <a class='anchor' id='prestudy'>Шаг 1. Изучение исходных данных</a>

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('/Users/Dom/Downloads/2bank_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
TOTAL_LINES = data.shape[0]
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод по шагу 1

Первый вывод исходя из типа данных в таблице:
- days_employed трудовой стаж в днях имеет формат float64, а число д.б. целым! Проверить!
- days_employed трудовой стаж days_employed - заполнен не для всех 21525 записей (есть пропуски, разобраться!)
- debt по логике в столбце (долги по кредиту) д.б. тип bool вместо int64
- total_income ежемес. доход указан не для всех записей - есть пропуски!
- ОШИБКА в children "-1" - исправить

## <a id='prep'>Шаг 2. Предобработка данных</a>

### 2.1. Обработка пропусков

In [5]:
### Обработка пропусков

data['days_employed'].isnull().sum()
# Всего пропуской в days_employed 2174 строк

2174

In [6]:
# Посмотрим на строки с пропусками, чтобы попытаться определить закономерности "на глаз"
data_employed_nan = data[data['days_employed'].isnull() == True]
data_employed_nan.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [7]:
data_employed_nan.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


In [12]:
# Проверим, совпадают ли строки с пропусками в колонке days_employed со строками в колонке total_income
data_total_income_nan = data[data['total_income'].isnull() == True]

try:
    data_nan_merged = data_employed_nan.merge(data_total_income_nan, on='children', how='left')
    print('Таблицы имеют одинаковые строки')
except:
    print('Таблицы имеют разные строки!!!')

# ---> Вывод сравнения: пропуски в обеих колонках - это одни и те же строки! И это хорошо!

Таблицы имеют одинаковые строки


In [13]:
# Для дальнейшего заполнения пропусков, сохраним средние значения колонок days_employed и total_income
days_employed_nonan = data[data['days_employed'].isnull() == False]
days_employed_nonan_mean = int(days_employed_nonan['days_employed'].mean()) # = 63046 (АХТУНГ, это более 172 ЛЕТ!!!)
total_income_nonan_mean = int(days_employed_nonan['total_income'].mean()) # = 167422

In [14]:
# Чтобы понять возможные причины появления пропусков в данных, попробуем поманипулировать таблицей data_employed_nan
data_employed_nan['education'].str.lower().value_counts()

среднее                1540
высшее                  544
неоконченное высшее      69
начальное                21
Name: education, dtype: int64

Больше всего пропуской у СОТРУДНИКОВ и КОМПАНЬОНОВ, т.е. предположение, что доход не указан в основном у пенсионеров и студентов (т.е. неработающих) - несостоятелен
После проверки по основным колонкам выявить какие-то закономерности "на глаз" не удалось.
Следовательно, делаем вывод, что причина появления пропусков - случайная ошибка в данных.

In [11]:
# Для начала заменим NaN на пустое значение ''
data['days_employed'] = data['days_employed'].fillna(value='')

# Сделаем то же самое для колонки total_income
data['total_income'] = data['total_income'].fillna(value='')


# Теперь заменим наши бывшие пропуски на средние значения в каждой колонке
data['days_employed'] = data['days_employed'].replace('', days_employed_nonan_mean)
data['total_income'] = data['total_income'].replace('', total_income_nonan_mean)

data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [12]:
# Проверим типы данных в колонках
data.info()
# -> Тип данных с колонках с пропусками изменился на object. Нужно вернуть int !!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод 2.1

1. определите и заполните пропущенные значения:
- опишите, какие пропущенные значения вы обнаружили;
- приведите возможные причины появления пропусков в данных;
- объясните, по какому принципу заполнены пропуски;

Пропущенные значения выявлены в колонках 'days_employed' и 'total_income'.

Всего таких строк 2174, т.е. 10,1% от общего числа 21525 строк. Это много.
Возможные причины появления пропусков в колонке days_employed -
    от возраста не зависит (распределение равномерное) + 101 строка с dob_years=0 - ОШИБКА!
    возможно зависит от образования (со средн.обр=1408+67+65=1540 eq 70,8% от пропусков, с высшим=23+25+496=544 eq 25% от пропусков)
    от пола (пропуски M=690, F=1484; всего M=7288, F=14236, одна строка gender='XNA' id 10701) - ОШИБКА!
    ...
Заполнение пропусков:
    для колонки трудовой стаж (дн) days_employed - на среднее по колонке
    для колонки ежемес.доход total_income - на среднее по колонке

### 2.2. Замена типа данных

In [13]:
# Используем метод .astype() для изменения типа данных
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод 2.2

Поскольку колонка days_employed содержит информацию о днях трудового стажа, то логично, что в ней должны содержаться ЦЕЛЫЕ числа.
До замены тип данных, содержащихся в колонке days_employed был float (нецелое число с плавающей точкой).
Поэтому заменил значения в столбце days_employed на int (целое число).

ДОПОЛНИТЕЛЬНО
поменяем тип данных в колонке total_income (доход в месяц) с float на int, поскольку информация "до копеек" тут не требуется.

Во всех остальных колонках тип данных корректный и замены не требует!

### 2.3. Обработка дубликатов

In [14]:
 # 4. Исправим регистры в education
data['education'] = data['education'].str.lower()
data['education'].value_counts()
# среднее=15233, высшее=5260, неоконченное высшее=744, начальное=282, уч.степень=6, ИТОГО-21525

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [15]:
# За одно 'для красоты' понизим регистр для столбца family_status
data['family_status'] = data['family_status'].str.lower()

In [16]:
# Посмотрим на строки, которые дублируются
data_dupl = data[data.duplicated(keep=False)]
# Посчитаем, сколько всего дублирующихся строк
data_dupl.count() # -> Всего 137 строк, из них 71 - повторы (keep='last')

children            137
days_employed       137
dob_years           137
education           137
education_id        137
family_status       137
family_status_id    137
gender              137
income_type         137
debt                137
total_income        137
purpose             137
dtype: int64

In [17]:
# применим drop_deplicates() совместно с reset_index() для удаления дубликатов
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated(keep=False).sum()
#data.head(15)

0

#### Вывод 2.3

Всего дубликатов методом .duplicated() выявлено 137 штуки (после правки регистра в education), из них повторов 71.

При просмотре выявлено, что это реально дублирующиеся строки, поэтому решено их удалить. 
Удаляю методом drop_duplicates().

Вероятная причина появления дубликатов - объединение нескольких баз, либо нескольких версий одной базы.

### 2.4. Лемматизация

In [18]:
from pymystem3 import Mystem
m = Mystem()
#data['lemmas'] = ' '.join(m.lemmatize(str(data['purpose'])))
# получаю список уникальных значений столбца purpose (причина для кредита)
purpose_unique = data['purpose'].unique()

# проверяю полученный список
#print(purpose_unique)

# создаю в цикле новый список уникальных лемм
lemmas = []
for i in purpose_unique:
#    result = m.lemmatize(i)
    result = ''.join(m.lemmatize(i)).strip()
    # Соединяем леммы друг с другом (вместо получения каждого слова как отдельного значения) [ ''.join() ],
    # а также удаляем все пробелы оператором .strip() 
    lemmas.append(result)
#print(lemmas)
# Получили Series с лемматизированными уникальными значениями всех строк таблицы data

#### Вывод 2.4

Типовых запросов не так уж много: жильё/недвижимость, автомобиль, свадьба, образование и другое

### 2.5. Исправление ошибок в данных

In [19]:
# 1. Поправим значение в количестве детей
data['children'] = data['children'].replace(-1, 1) # предполагаю, что это ошибка ввода, т.к. единичный случай
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

In [20]:
# 6. Исправим значение в gender 
data['gender'] = data['gender'].replace('XNA', 'F') # меняю на 'F' т.к. по статистике их в два раза больше
data['gender'].value_counts()

F    14175
M     7279
Name: gender, dtype: int64

In [21]:
# 3. Исправим значения в dob_years (возраст заёмщика)
# Найтём средний возраст для тех, у кого он указан, запишем его в переменную dob_years_mean
data_age_nozero = data[data['dob_years'] != 0]
dob_years_mean = data_age_nozero['dob_years'].mean()       # результат 43.4974
dob_years_mean = int(data_age_nozero['dob_years'].mean()) # результат 43
print(dob_years_mean)

43


In [22]:
# Заменим возраст 0 на средний возраст в таблице
data['dob_years'] = data['dob_years'].replace(0, dob_years_mean)
data['dob_years'].value_counts()
data[data['dob_years'] == 0]['family_status'].count()

0

In [23]:
                                    # 2. Поправим значения в трудовом стаже days_employed
## Предполагаю, что отрицательные значения - ошибка
# Проверим, сколько всего строк с отрицательным days_employed
#data[data['days_employed'] < 0]['days_employed'].count()
# Результат: всего 15906 строк с отрицательным days_employed (т.е. почти 74%) Это ОЧЕНЬ Много!!!
# очевидно присутствует какая-то системная ошибка

# Проверим, как распределяется по возрастам
#data[data['days_employed'] < 0]['dob_years'].value_counts()
# Результат: Распределение более-менее равномерное

# Проверим, как распределяется по типу образования
#data[data['days_employed'] < 0]['education'].value_counts()
# Результат: среднее=10899, высшее=419, неоконч.высшее=640, начальное=168, уч.степень=4
# Вывод: Строгой зависимости нет

# Проверим, как распределяется по профессии income_type
#data[data['days_employed'] < 0]['income_type'].value_counts()
# Результат: сотрудник=10014, компаньон=4577, госслужащий=1312, предприниматель=1, в.декрете=1, студент=1
# Вывод: Строгой зависимости нет

# Проверим диапазон отрицательных значений
#data[data['days_employed'] < 0].sort_values(by='days_employed').head()
#data[data['days_employed'] < 0].sort_values(by='days_employed').tail(50)
# Результат: Отрицательные значения в диапазоне от -18388 до -24

#data_minus = data[data['days_employed'] < -80]
#data_minus[data_minus['days_employed'] > -90].sort_values(by='days_employed')

# Проверим, если ли связь отрицательных значений стажа days_employed с неотдачей кредита
#data[data['days_employed'] < 0]['debt'].value_counts()
# Результат: 14517 - не имело задолженности, 1389 - имело. Вывод: Связи нет!
#data['days_employed'].min()
### ОБЩИЙ ВЫВОД по столбцу days_employed: Значения неадекватные. В расчёт не принимать!!!


### 2.6. Категоризация данных

In [24]:
# Выделим 4 категории в зависимости от цели получения кредита (и создадим одну для проверки)

# Альтернативный вариант функции БЕЗ обращения к новому (несозданному) столбцу 
def purpose_type(row):
    """
    Определяет тип причины для кредита исходя из условий:
    ключевое слово "жилье" или "недвижимость", то тип "недвижимость"
    ключ.сл. "автомобиль", то тип "авто"
    "образование", то "образование"
    "свадьба", то "свадьба"
    иначе, "другое"
    """
    lemmatized_purpose = m.lemmatize(row['purpose'])
    if 'жилье' in lemmatized_purpose or 'недвижимость' in lemmatized_purpose:
        return 'недвижимость'
    if 'автомобиль' in lemmatized_purpose:
        return 'авто'
    if 'образование' in lemmatized_purpose:
        return 'образование'
    if 'свадьба' in lemmatized_purpose:
        return 'свадьба'
    else:
        return 'прочее' # создаю на всякий случай как проверку

In [25]:
# Проверим зависимость возврата от пола
pivot_gender_debt = data.pivot_table(index='debt', columns='gender',values='income_type', aggfunc='count').T
pivot_gender_debt['perc_of_tot'] = pivot_gender_debt[1]/(pivot_gender_debt[0]+pivot_gender_debt[1])*100
pivot_gender_debt

debt,0,1,perc_of_tot
gender,,,
F,13181,994,7.012346
M,6532,747,10.262399


In [26]:
# Проверим зависимость возврата от профессии
pivot_income_type_debt = data.pivot_table(index='debt', columns='income_type',values='gender', aggfunc='count').T
pivot_income_type_debt['perc_of_tot'] = pivot_income_type_debt[1]/(pivot_income_type_debt[0]+pivot_income_type_debt[1])*100
pivot_income_type_debt.fillna(0).sort_values(by='perc_of_tot', ascending=False)
# Процент невозврата  безработный - 50%, в декрете - 100%, госслужащий - 6%, компаньон - 7%, пенсионер - 5,6%,
# предприниматель - 0%, сотрудник - 9,5%, студент - 0%
# при этом студентам, предпринимателям, безработным и в декрете - кредиты почти не выдают!!!

debt,0,1,perc_of_tot
income_type,,,
безработный,1.0,1.0,50.000000
сотрудник,10023.0,1061.0,9.572357
компаньон,4702.0,376.0,7.404490
госслужащий,1371.0,86.0,5.902539
пенсионер,3613.0,216.0,5.641160
в декрете,0.0,1.0,0.000000
предприниматель,2.0,0.0,0.000000
студент,1.0,0.0,0.000000


In [27]:
# Пропишем для каждого наблюдения тип причины для кредита
data['purpose_type'] = data.apply(purpose_type, axis='columns') #alt 'axis=1'

In [28]:
# Проверка назначения типа
data['purpose_type'].unique()

array(['недвижимость', 'авто', 'образование', 'свадьба'], dtype=object)

In [29]:
# Посчитаем, сколько запросов в каждой категории
display(data['purpose_type'].value_counts().to_frame())
# В процентах
display(data['purpose_type'].value_counts(normalize=True).to_frame())

# Результат недвижимость=10840 (eq. 50,4%), авто=4315 (eq. 20%), образование=4022 (eq. 18,7%), 
#           свадьба=2348 (eq. 10,9%), ВСЕГО=21525

,purpose_type
недвижимость,10811
авто,4306
образование,4013
свадьба,2324


,purpose_type
недвижимость,0.503915
авто,0.200708
образование,0.187051
свадьба,0.108325


### Вывод по шагу 2

В итоге получили 4 категории причин получения кредита: 
- недвижимость (50,4% или 10840 запросов)
- авто (20,0% или 4315 запросов)
- образование (18,7% или 4022 запросов)
- свадьба (10,9% или 2348 запросов)


## <a class='anchor' id='analysis'>Шаг 3. Анализ данных</a>

### Вопрос 3.1: Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
# Проверим, если ли зависимость, для этого создадим сводную таблицу
pivot_children_debt = data.pivot_table(index='debt', columns='children',values='gender', aggfunc='count')
pivot_children_debt.head(35)
# Результат:
    # без детей: нет долгов = 13086 (60% от 21525) всего бездетных 14149(92% от бездетных)
    #            есть долги = 1063 (5% от 21525) (8% от бездетных)
    # C детьми: нет долгов = 4420+1861+303+37+9+68=6698 (31% от 21525) всего с-детьми 7376 (91% от с-детьми)
    #            есть долги = 445+194+27+4+0+8=678 (3% от 21525) (9% от с-детьми)

children,0,1,2,3,4,5,20
debt,,,,,,,
0,13028.0,4410.0,1858.0,303.0,37.0,9.0,68.0
1,1063.0,445.0,194.0,27.0,4.0,NaN,8.0


#### Ответ 3.1

Корреляции между наличием детей и возвратом кредита нет (процент возврата примерно одинаковый).

Результат:
* без детей: 
    * нет долгов = 13086 (60% от 21525) всего бездетных 14149(92% от бездетных)
    * есть долги = 1063 (5% от 21525) (8% от бездетных)
* с детьми: 
    * нет долгов = 4420+1861+303+37+9+68=6698 (31% от 21525) всего с-детьми 7376 (91% от с-детьми)
    * есть долги = 445+194+27+4+0+8=678 (3% от 21525) (9% от с-детьми)

### Вопрос 3.2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
# Проверим, если ли зависимость, для этого создадим сводную таблицу
pivot_famstat_debt = data.pivot_table(index='debt', columns='family_status',values='gender', aggfunc='count').T
pivot_famstat_debt['1_perc_of_tot'] = pivot_famstat_debt[1]/(pivot_famstat_debt[0]+pivot_famstat_debt[1])*100
pivot_famstat_debt.sort_values(by='1_perc_of_tot', ascending=False)
# Результат:
    # не женат: нет долгов = 2539; всего холостых 2813(90% от холостых)
    #           есть долги = 274; (10% от холостых)
    # в разводе: нет долгов = 1110; всего в-разводе 1195 (93% от в-разводе)
    #            есть долги = 85; (7% от в-разводе)
    # вдова:     нет долгов = 897; всего вдова 960 (93% от вдова)
    #            есть долги = 63; (7% от вдова)
    # гр.брак:   нет долгов = 3789; всего гр.брак 4177 (91% от гр.брак)
    #            есть долги = 388; (9% от гр.брак)
    # женат:     нет долгов = 11449; всего женат 12380 (92% от женат)
    #            есть долги = 931; (8% от женат)

debt,0,1,1_perc_of_tot
family_status,,,
не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


#### Ответ 3.2

Слабая корреляции между семейным положением и возвратом кредита отслеживается.

Результат:
* не женат: 
    * нет долгов = 2539; всего холостых 2813(90% от холостых)
    * есть долги = 274; (10% от холостых)
* в разводе: 
    * нет долгов = 1110; всего в-разводе 1195 (93% от в-разводе)
    * есть долги = 85; (7% от в-разводе)
* вдова:
    * нет долгов = 897; всего вдова 960 (93% от вдова)
    * есть долги = 63; (7% от вдова)
* гр.брак:  
    * нет долгов = 3789; всего гр.брак 4177 (91% от гр.брак)
    * есть долги = 388; (9% от гр.брак)
* женат:     
    * нет долгов = 11449; всего женат 12380 (92% от женат)
    * есть долги = 931; (8% от женат)
    
То есть <u>самые надежные заемщики - в разводе, вдовы и женатые</u><br>
Самые ненадежные заемщики - холостые и в гражданском браке

### Вопрос 3.3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
# Сначала разделим на статусы в зависимости от заработка
#data.sort_values(by='total_income').tail()
# минимальное значение 20'667
# максимальное значение 2'265'604
def income_status(row):
    '''
    Определяет статус исходя из уровня заработка
    До 50т.р. - минимум
    менее 100т.р. - ниже среднего
    менее 300т.р. - средний
    менее 1млн.р. - выше среднего
    более 1млн.р. - максимум
    '''
    total_income = row['total_income']
    if total_income < 50000: # доход менее 50тр - минимум
        return 'минимум'
    if total_income < 100000: # доход от 50тр до 100тр - ниже среднего
        return 'ниже среднего'
    if total_income < 200000: # доход от 100тр до 200тр - средний
        return 'средний'
    if total_income < 500000: # доход от 200тр до 500тр - выше среднего
        return 'выше среднего'
    else:
        return 'максимум' # доход более 500тр в месяц - максимум

In [33]:
data['total_income_status'] = data.apply(income_status, axis='columns')
#data.head()

# Создадим сводную таблицу для проверки зависимости
pivot_income_debt = data.pivot_table(index=['debt'], columns='total_income_status',values='gender', aggfunc='count').T
pivot_income_debt['total'] = pivot_income_debt[0]+pivot_income_debt[1]
pivot_income_debt['1_perc_of_tot'] = pivot_income_debt[1]/pivot_income_debt['total']*100
pivot_income_debt.sort_values(by='1_perc_of_tot', ascending=False)

debt,0,1,total,1_perc_of_tot
total_income_status,,,,
средний,10896,1029,11925,8.628931
ниже среднего,3760,331,4091,8.090931
выше среднего,4500,344,4844,7.101569
максимум,208,14,222,6.306306
минимум,349,23,372,6.182796


#### Ответ 3.3

Очень небольшая зависимость возврата кредита от уровня дохода есть:
* Самый высокий процент невозврата:
* 9% в группе со средними доходами (от 100тр до 200тр)
* 8% в группе с доходами ниже среднего (от 50тр дл 100тр)

<p>Самый низкий процент невозврата в группе с доходами менее 50тр - всего 6% (хотя и число выданных кредитов сильно меньше, чем в остальных группах).</p>

### Вопрос 3.4: Как разные цели кредита влияют на его возврат в срок?

In [34]:
# Создадим сводную таблицу для проверки зависимости
pivot_purpose_debt = data.pivot_table(index=['debt'], columns='purpose_type',values='gender', aggfunc='count')
pivot_purpose_debt.head()

purpose_type,авто,недвижимость,образование,свадьба
debt,,,,
0,3903,10029,3643,2138
1,403,782,370,186


#### Ответ 3.4

Результаты:
<p>Невозврат по группам, %<br>
* Авто 9% (3912-403)<br>
* Недвижимость 7% (10058-782)<br>
* Образование 9%(3652-370)<br>
* Свадьба 8% (2162-186)<br>
    
<b>Вывод: Самая надежная цель для получения кредита - категория "недвижимость".<br>
Самая ненадежная цель - "авто" и (как ни странно) "образование"</b>

## <a class='anchor' id='resume'>Шаг 4. Общий вывод</a>

Возврат выданного кредита зависит:
    - от семейного положение (самые надежные заемщики - это "в разводе", "вдовы/вдовцы" и "женатые/замужем", а самые ненадежные заемщики - "холостые" и "в гражданском браке"
    - от цели кредита (самые надежные заемщики берут кредит на покупку или ремонт недвижимости, а самые ненадежные - на автомобиль и образование.
Возврат выданного кредита не зависит:
    - от наличия детей (возвращают одинаково и заемщики с детьми, и бездетные)
    - от уровня дохода (заемщики с разным уровнем дохода имеют примерно одинаковый процент возврата кредита в срок)
    
Таким образом, портрет идеального заемщика: Женщина-пенсионер или госслужащая, состоящая в браке, которая планирует приобретение недвижимости.
Потртет нежелательного заемщика: Холостой мужчина, сотрудник по найму, которых хочет взять кредит на автомобиль.